In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

from custom_models import get_cell_based_tiny_net

# NB201
from nas_201_api import NASBench201API as API

2022-09-14 05:57:14.806160: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
parser = argparse.ArgumentParser("Random search for NAS.")
parser.add_argument("--data_path", type=str, default='../cifar.python', help="The path to dataset")
parser.add_argument("--dataset", type=str, default='cifar10',choices=["cifar10", "cifar100", "ImageNet16-120"], help="Choose between Cifar10/100 and ImageNet-16.")

# channels and number-of-cells
parser.add_argument("--search_space_name", type=str, default='nas-bench-201', help="The search space name.")
parser.add_argument("--config_path", type=str, default='../configs/nas-benchmark/algos/RANDOM.config', help="The path to the configuration.")
parser.add_argument("--max_nodes", type=int, default=4, help="The maximum number of nodes.")
parser.add_argument("--channel", type=int, default=16, help="The number of channels.")
parser.add_argument("--num_cells", type=int, default=5, help="The number of cells in one stage.")
parser.add_argument("--select_num", type=int, default=100, help="The number of selected architectures to evaluate.")
parser.add_argument("--track_running_stats", type=int, default=0, choices=[0, 1], help="Whether use track_running_stats or not in the BN layer.")
# log
parser.add_argument("--workers", type=int, default=4, help="number of data loading workers")
parser.add_argument("--save_dir", type=str, default='../results/NB201/Cell-wise-RSPS-CIFAR10-BN0', help="Folder to save checkpoints and log.")
# parser.add_argument("--arch_nas_dataset", type=str, default='../NAS-Bench-201-v1_1-096897.pth', help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--arch_nas_dataset", type=str, default=None, help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--print_freq", type=int, default=100, help="print frequency (default: 200)")
parser.add_argument("--rand_seed", type=int, default=None, help="manual seed")
args = parser.parse_args(args=[])
if args.rand_seed is None or args.rand_seed < 0:
    args.rand_seed = random.randint(1, 100000)

    
print(args.rand_seed)
print(args)
xargs=args

16467
Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=100, rand_seed=16467, save_dir='../results/NB201/Cell-wise-RSPS-CIFAR10-BN0', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [3]:
assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads(xargs.workers)
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

Main Function with logger : Logger(dir=../results/NB201/Cell-wise-RSPS-CIFAR10-BN0, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ../configs/RANDOM.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 100
rand_seed        : 16467
save_dir         : ../results/NB201/Cell-wise-RSPS-CIFAR10-BN0
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None


In [4]:
def search_func(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        ### sample a cell-wise randomized network
        network.module.random_genotype_per_cell(True)
        ###
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg


def valid_func(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

            network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg


def search_find_best(xloader, network, n_samples):
    with torch.no_grad():
        network.eval()
        archs, valid_accs = [], []
        # print ('obtain the top-{:} architectures'.format(n_samples))
        loader_iter = iter(xloader)
        for i in range(n_samples):
            arch = network.module.random_genotype_per_cell(True)
            try:
                inputs, targets = next(loader_iter)
            except:
                loader_iter = iter(xloader)
                inputs, targets = next(loader_iter)

            _, logits = network(inputs)
            val_top1, val_top5 = obtain_accuracy(
                logits.cpu().data, targets.data, topk=(1, 5)
            )

            archs.append(arch)
            valid_accs.append(val_top1.item())

        best_idx = np.argmax(valid_accs)
        best_arch, best_valid_acc = archs[best_idx], valid_accs[best_idx]
        return best_arch, best_valid_acc

In [5]:
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(xargs.config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.test_batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": bool(xargs.track_running_stats),
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log("{:} create API = {:} done".format(time_string(), api))

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists():  # automatically resume from previous checkpoint
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start".format(last_info)
    )
    last_info = torch.load(last_info)
    start_epoch = last_info["epoch"]
    checkpoint = torch.load(last_info["last_checkpoint"])
    genotypes = checkpoint["genotypes"]
    valid_accuracies = checkpoint["valid_accuracies"]
    search_model.load_state_dict(checkpoint["search_model"])
    w_scheduler.load_state_dict(checkpoint["w_scheduler"])
    w_optimizer.load_state_dict(checkpoint["w_optimizer"])
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(
            last_info, start_epoch
        )
    )
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Files already downloaded and verified
Files already downloaded and verified
../configs/RANDOM.config
Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=250, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
||||||| cifar10    ||||||| Search-Loader-Num=391, Valid-Loader-Num=49, batch size=64
||||||| cifar10    ||||||| Config=Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=250, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
w-optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.025
    lr: 0.025
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)
w-scheduler : CosineAnnealingLR(warmup=0, max-epoch=250, current::epoch=0, iter=0.00, type=cosine, T-max=250, eta-min=0.001)
criterion   : 

In [ ]:
# start training
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func(
        search_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func(
        valid_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    cur_arch, cur_valid_acc = search_find_best(
        valid_loader, network, xargs.select_num
    )
    logger.log(
        "[{:}] find-the-best : {:}, accuracy@1={:.2f}%".format(
            epoch_str, cur_arch, cur_valid_acc
        )
    )
    genotypes[epoch] = cur_arch
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log("\n" + "-" * 200)
logger.log("Pre-searching costs {:.1f} s".format(search_time.sum))
start_time = time.time()
best_arch, best_acc = search_find_best(valid_loader, network, xargs.select_num)
search_time.update(time.time() - start_time)
logger.log(
    "RANDOM-NAS finds the best one : {:} with accuracy={:.2f}%, with {:.1f} s.".format(
        best_arch, best_acc, search_time.sum
    )
)
if api is not None:
    logger.log("{:}".format(api.query_by_arch(best_arch, "200")))
logger.close()


[Search the 000-250-th epoch] Time Left: [00:00:00], LR=0.025
*SEARCH* [2022-09-14 05:57:21] [000-250][000/391] Time 2.93 (2.93) Data 0.17 (0.17) Base [Loss 2.360 (2.360)  Prec@1 3.12 (3.12) Prec@5 51.56 (51.56)]
*SEARCH* [2022-09-14 05:57:34] [000-250][100/391] Time 0.12 (0.16) Data 0.00 (0.00) Base [Loss 2.279 (2.315)  Prec@1 10.94 (11.22) Prec@5 59.38 (49.78)]
*SEARCH* [2022-09-14 05:57:47] [000-250][200/391] Time 0.12 (0.15) Data 0.00 (0.00) Base [Loss 2.321 (2.310)  Prec@1 10.94 (11.10) Prec@5 46.88 (50.87)]
*SEARCH* [2022-09-14 05:58:00] [000-250][300/391] Time 0.12 (0.14) Data 0.00 (0.00) Base [Loss 2.297 (2.307)  Prec@1 18.75 (11.19) Prec@5 57.81 (51.32)]
*SEARCH* [2022-09-14 05:58:12] [000-250][390/391] Time 0.12 (0.14) Data 0.00 (0.00) Base [Loss 2.364 (2.302)  Prec@1 7.50 (11.39) Prec@5 45.00 (52.09)]
[000-250] searching : loss=2.30, accuracy@1=11.39%, accuracy@5=52.09%, time-cost=54.2 s
[000-250] evaluate  : loss=2.32, accuracy@1=12.03%, accuracy@5=54.38%
[000-250] find-th

*SEARCH* [2022-09-14 06:00:13] [003-250][000/391] Time 0.45 (0.45) Data 0.29 (0.29) Base [Loss 2.287 (2.287)  Prec@1 12.50 (12.50) Prec@5 53.12 (53.12)]
*SEARCH* [2022-09-14 06:00:22] [003-250][100/391] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 2.228 (2.273)  Prec@1 15.62 (12.78) Prec@5 59.38 (56.03)]
*SEARCH* [2022-09-14 06:00:31] [003-250][200/391] Time 0.08 (0.09) Data 0.00 (0.00) Base [Loss 2.316 (2.268)  Prec@1 9.38 (12.66) Prec@5 45.31 (57.18)]
*SEARCH* [2022-09-14 06:00:44] [003-250][300/391] Time 0.12 (0.10) Data 0.00 (0.00) Base [Loss 2.192 (2.267)  Prec@1 12.50 (12.73) Prec@5 67.19 (57.14)]
*SEARCH* [2022-09-14 06:00:54] [003-250][390/391] Time 0.19 (0.11) Data 0.00 (0.00) Base [Loss 2.241 (2.265)  Prec@1 17.50 (12.82) Prec@5 70.00 (57.40)]
[003-250] searching : loss=2.26, accuracy@1=12.82%, accuracy@5=57.40%, time-cost=172.7 s
[003-250] evaluate  : loss=2.28, accuracy@1=12.79%, accuracy@5=57.20%
[003-250] find-the-best : [Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_

*SEARCH* [2022-09-14 06:03:00] [006-250][000/391] Time 0.38 (0.38) Data 0.28 (0.28) Base [Loss 2.304 (2.304)  Prec@1 9.38 (9.38) Prec@5 40.62 (40.62)]
*SEARCH* [2022-09-14 06:03:09] [006-250][100/391] Time 0.14 (0.09) Data 0.00 (0.00) Base [Loss 2.254 (2.231)  Prec@1 12.50 (13.71) Prec@5 60.94 (60.86)]
*SEARCH* [2022-09-14 06:03:23] [006-250][200/391] Time 0.13 (0.12) Data 0.00 (0.00) Base [Loss 2.202 (2.230)  Prec@1 14.06 (14.18) Prec@5 62.50 (60.97)]
*SEARCH* [2022-09-14 06:03:38] [006-250][300/391] Time 0.13 (0.12) Data 0.00 (0.00) Base [Loss 2.299 (2.227)  Prec@1 6.25 (14.51) Prec@5 48.44 (61.44)]
*SEARCH* [2022-09-14 06:03:47] [006-250][390/391] Time 0.08 (0.12) Data 0.00 (0.00) Base [Loss 2.029 (2.222)  Prec@1 22.50 (14.74) Prec@5 80.00 (62.04)]
[006-250] searching : loss=2.22, accuracy@1=14.74%, accuracy@5=62.04%, time-cost=302.3 s
[006-250] evaluate  : loss=2.25, accuracy@1=13.99%, accuracy@5=60.91%
[006-250] find-the-best : [Structure(4 nodes with |skip_connect~0|+|nor_conv_1x

*SEARCH* [2022-09-14 06:05:51] [009-250][000/391] Time 0.40 (0.40) Data 0.31 (0.31) Base [Loss 2.303 (2.303)  Prec@1 10.94 (10.94) Prec@5 51.56 (51.56)]
*SEARCH* [2022-09-14 06:06:03] [009-250][100/391] Time 0.12 (0.13) Data 0.00 (0.00) Base [Loss 2.165 (2.202)  Prec@1 15.62 (15.75) Prec@5 67.19 (64.02)]
*SEARCH* [2022-09-14 06:06:15] [009-250][200/391] Time 0.07 (0.12) Data 0.00 (0.00) Base [Loss 2.292 (2.201)  Prec@1 10.94 (15.71) Prec@5 50.00 (63.95)]
*SEARCH* [2022-09-14 06:06:23] [009-250][300/391] Time 0.07 (0.11) Data 0.00 (0.00) Base [Loss 2.321 (2.203)  Prec@1 10.94 (15.85) Prec@5 53.12 (63.85)]
*SEARCH* [2022-09-14 06:06:30] [009-250][390/391] Time 0.07 (0.10) Data 0.00 (0.00) Base [Loss 2.129 (2.198)  Prec@1 15.00 (15.96) Prec@5 62.50 (64.17)]
[009-250] searching : loss=2.20, accuracy@1=15.96%, accuracy@5=64.17%, time-cost=422.2 s
[009-250] evaluate  : loss=2.27, accuracy@1=13.37%, accuracy@5=57.46%
[009-250] find-the-best : [Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv

*SEARCH* [2022-09-14 06:08:37] [012-250][000/391] Time 0.37 (0.37) Data 0.28 (0.28) Base [Loss 2.170 (2.170)  Prec@1 12.50 (12.50) Prec@5 64.06 (64.06)]
*SEARCH* [2022-09-14 06:08:46] [012-250][100/391] Time 0.07 (0.09) Data 0.00 (0.00) Base [Loss 2.166 (2.188)  Prec@1 15.62 (17.02) Prec@5 70.31 (64.82)]
*SEARCH* [2022-09-14 06:08:54] [012-250][200/391] Time 0.07 (0.08) Data 0.00 (0.00) Base [Loss 2.301 (2.194)  Prec@1 7.81 (16.22) Prec@5 60.94 (64.66)]
*SEARCH* [2022-09-14 06:09:07] [012-250][300/391] Time 0.13 (0.10) Data 0.00 (0.00) Base [Loss 2.198 (2.193)  Prec@1 25.00 (16.50) Prec@5 75.00 (64.94)]
*SEARCH* [2022-09-14 06:09:20] [012-250][390/391] Time 0.12 (0.11) Data 0.00 (0.00) Base [Loss 2.350 (2.190)  Prec@1 10.00 (16.52) Prec@5 62.50 (65.06)]
[012-250] searching : loss=2.19, accuracy@1=16.52%, accuracy@5=65.06%, time-cost=550.3 s
[012-250] evaluate  : loss=2.26, accuracy@1=16.60%, accuracy@5=66.15%
[012-250] find-the-best : [Structure(4 nodes with |skip_connect~0|+|avg_pool_

...

*SEARCH* [2022-09-14 09:12:43] [199-250][000/391] Time 0.52 (0.52) Data 0.26 (0.26) Base [Loss 1.087 (1.087)  Prec@1 57.81 (57.81) Prec@5 93.75 (93.75)]
*SEARCH* [2022-09-14 09:12:58] [199-250][100/391] Time 0.22 (0.15) Data 0.00 (0.00) Base [Loss 1.082 (1.522)  Prec@1 64.06 (43.39) Prec@5 100.00 (83.28)]
*SEARCH* [2022-09-14 09:13:13] [199-250][200/391] Time 0.14 (0.15) Data 0.00 (0.00) Base [Loss 2.303 (1.565)  Prec@1 7.81 (41.96) Prec@5 57.81 (81.48)]
*SEARCH* [2022-09-14 09:13:28] [199-250][300/391] Time 0.14 (0.15) Data 0.00 (0.00) Base [Loss 1.193 (1.546)  Prec@1 51.56 (42.55) Prec@5 95.31 (81.94)]
*SEARCH* [2022-09-14 09:13:41] [199-250][390/391] Time 0.14 (0.15) Data 0.00 (0.00) Base [Loss 1.367 (1.539)  Prec@1 52.50 (42.78) Prec@5 95.00 (82.20)]
[199-250] searching : loss=1.54, accuracy@1=42.78%, accuracy@5=82.20%, time-cost=8988.3 s
[199-250] evaluate  : loss=1.75, accuracy@1=40.12%, accuracy@5=81.67%
[199-250] find-the-best : [Structure(4 nodes with |nor_conv_1x1~0|+|nor_con

*SEARCH* [2022-09-14 09:15:58] [202-250][000/391] Time 0.37 (0.37) Data 0.26 (0.26) Base [Loss 1.157 (1.157)  Prec@1 57.81 (57.81) Prec@5 96.88 (96.88)]
*SEARCH* [2022-09-14 09:16:10] [202-250][100/391] Time 0.14 (0.12) Data 0.00 (0.00) Base [Loss 1.134 (1.595)  Prec@1 59.38 (40.97) Prec@5 96.88 (79.87)]
*SEARCH* [2022-09-14 09:16:25] [202-250][200/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 1.227 (1.556)  Prec@1 57.81 (42.18) Prec@5 95.31 (81.23)]
*SEARCH* [2022-09-14 09:16:40] [202-250][300/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 1.118 (1.531)  Prec@1 60.94 (43.53) Prec@5 96.88 (82.14)]
*SEARCH* [2022-09-14 09:16:54] [202-250][390/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 1.231 (1.548)  Prec@1 55.00 (42.64) Prec@5 97.50 (81.56)]
[202-250] searching : loss=1.55, accuracy@1=42.64%, accuracy@5=81.56%, time-cost=9139.0 s
[202-250] evaluate  : loss=1.76, accuracy@1=40.38%, accuracy@5=80.83%
[202-250] find-the-best : [Structure(4 nodes with |skip_connect~0|+|avg_poo

*SEARCH* [2022-09-14 09:19:03] [205-250][000/391] Time 0.49 (0.49) Data 0.32 (0.32) Base [Loss 1.245 (1.245)  Prec@1 56.25 (56.25) Prec@5 96.88 (96.88)]
*SEARCH* [2022-09-14 09:19:17] [205-250][100/391] Time 0.13 (0.14) Data 0.00 (0.00) Base [Loss 2.305 (1.501)  Prec@1 7.81 (44.00) Prec@5 48.44 (82.77)]
*SEARCH* [2022-09-14 09:19:28] [205-250][200/391] Time 0.13 (0.13) Data 0.00 (0.00) Base [Loss 1.358 (1.484)  Prec@1 50.00 (45.29) Prec@5 95.31 (84.20)]
*SEARCH* [2022-09-14 09:19:39] [205-250][300/391] Time 0.08 (0.12) Data 0.00 (0.00) Base [Loss 1.438 (1.504)  Prec@1 48.44 (44.23) Prec@5 92.19 (83.40)]
*SEARCH* [2022-09-14 09:19:47] [205-250][390/391] Time 0.08 (0.12) Data 0.00 (0.00) Base [Loss 2.297 (1.508)  Prec@1 17.50 (44.02) Prec@5 45.00 (83.22)]
[205-250] searching : loss=1.51, accuracy@1=44.02%, accuracy@5=83.22%, time-cost=9272.1 s
[205-250] evaluate  : loss=1.79, accuracy@1=38.73%, accuracy@5=79.37%
[205-250] find-the-best : [Structure(4 nodes with |nor_conv_3x3~0|+|none~0|n

*SEARCH* [2022-09-14 09:21:49] [208-250][000/391] Time 0.45 (0.45) Data 0.30 (0.30) Base [Loss 1.048 (1.048)  Prec@1 59.38 (59.38) Prec@5 95.31 (95.31)]
*SEARCH* [2022-09-14 09:22:03] [208-250][100/391] Time 0.12 (0.14) Data 0.00 (0.00) Base [Loss 1.216 (1.488)  Prec@1 54.69 (45.65) Prec@5 93.75 (83.99)]
*SEARCH* [2022-09-14 09:22:16] [208-250][200/391] Time 0.12 (0.14) Data 0.00 (0.00) Base [Loss 1.172 (1.541)  Prec@1 64.06 (43.00) Prec@5 96.88 (81.54)]
*SEARCH* [2022-09-14 09:22:29] [208-250][300/391] Time 0.12 (0.13) Data 0.00 (0.00) Base [Loss 1.156 (1.528)  Prec@1 54.69 (43.35) Prec@5 93.75 (82.22)]
*SEARCH* [2022-09-14 09:22:41] [208-250][390/391] Time 0.12 (0.13) Data 0.00 (0.00) Base [Loss 1.383 (1.512)  Prec@1 52.50 (44.04) Prec@5 92.50 (82.90)]
[208-250] searching : loss=1.51, accuracy@1=44.04%, accuracy@5=82.90%, time-cost=9404.3 s
[208-250] evaluate  : loss=1.73, accuracy@1=40.28%, accuracy@5=80.78%
[208-250] find-the-best : [Structure(4 nodes with |nor_conv_3x3~0|+|nor_con

*SEARCH* [2022-09-14 09:24:44] [211-250][000/391] Time 0.43 (0.43) Data 0.27 (0.27) Base [Loss 1.245 (1.245)  Prec@1 50.00 (50.00) Prec@5 95.31 (95.31)]
*SEARCH* [2022-09-14 09:24:53] [211-250][100/391] Time 0.09 (0.09) Data 0.00 (0.00) Base [Loss 1.446 (1.438)  Prec@1 57.81 (47.01) Prec@5 90.62 (84.79)]
*SEARCH* [2022-09-14 09:25:05] [211-250][200/391] Time 0.14 (0.11) Data 0.00 (0.00) Base [Loss 1.119 (1.495)  Prec@1 62.50 (44.77) Prec@5 95.31 (82.45)]
*SEARCH* [2022-09-14 09:25:20] [211-250][300/391] Time 0.14 (0.12) Data 0.00 (0.00) Base [Loss 2.305 (1.511)  Prec@1 7.81 (43.79) Prec@5 48.44 (81.71)]
*SEARCH* [2022-09-14 09:25:32] [211-250][390/391] Time 0.07 (0.13) Data 0.00 (0.00) Base [Loss 1.236 (1.520)  Prec@1 55.00 (43.40) Prec@5 97.50 (81.43)]
[211-250] searching : loss=1.52, accuracy@1=43.40%, accuracy@5=81.43%, time-cost=9529.6 s
[211-250] evaluate  : loss=1.74, accuracy@1=39.96%, accuracy@5=79.19%
[211-250] find-the-best : [Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv

# Train a found model

In [14]:
print(best_arch)
import copy
best_arch_copy = copy.deepcopy(best_arch)

[Structure(4 nodes with |nor_conv_3x3~0|+|avg_pool_3x3~0|skip_connect~1|+|skip_connect~0|nor_conv_1x1~1|none~2|), Structure(4 nodes with |avg_pool_3x3~0|+|nor_conv_3x3~0|none~1|+|avg_pool_3x3~0|none~1|skip_connect~2|), Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|avg_pool_3x3~0|nor_conv_1x1~1|avg_pool_3x3~2|), Structure(4 nodes with |nor_conv_1x1~0|+|avg_pool_3x3~0|avg_pool_3x3~1|+|avg_pool_3x3~0|nor_conv_3x3~1|none~2|), Structure(4 nodes with |nor_conv_3x3~0|+|skip_connect~0|nor_conv_3x3~1|+|skip_connect~0|skip_connect~1|nor_conv_1x1~2|), Structure(4 nodes with |none~0|+|avg_pool_3x3~0|none~1|+|nor_conv_1x1~0|nor_conv_3x3~1|skip_connect~2|), Structure(4 nodes with |avg_pool_3x3~0|+|none~0|none~1|+|avg_pool_3x3~0|skip_connect~1|none~2|), Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|avg_pool_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|), Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|nor_conv_3x3~0|nor_conv_1x1~1|

In [15]:
print(args)
args.save_dir = '../results/NB201/test'

Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=100, rand_seed=16467, save_dir='../results/NB201/test', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [17]:
print(config)

Configure(scheduler='cos', eta_min=0.0, epochs=200, warmup=0, optim='SGD', LR=0.1, decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=256, class_num=10, xshape=(1, 3, 32, 32))


In [18]:
logger = prepare_logger(args)

cifar_train_config_path = "../configs/nas-benchmark/CIFAR.config"
###
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(cifar_train_config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": True, # true for eval
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))

network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)

start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Main Function with logger : Logger(dir=../results/NB201/test, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ../configs/RANDOM.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 100
rand_seed        : 16467
save_dir         : ../results/NB201/test
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloaded and verified
../configs/nas-benchmark/CIFAR.config
Configure(scheduler='cos', eta_min=0.0, epochs=200, warmup=0, optim='SGD', LR=0.1, decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', b

In [20]:
from custom_search_cells import NAS201SearchCell as SearchCell

i = 0
for cell in network.module.cells:
    if isinstance(cell, SearchCell):
        cell.arch_cache = best_arch[i]
        i = i+1
print(i)
print(len(best_arch))

15
15


In [21]:
def search_func_one_arch(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        ### sample a cell-wise randomized network
#         network.module.random_genotype_per_cell(True)
        ###
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg

def valid_func_one_arch(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

#             network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg

In [22]:
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(0, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func_one_arch(
        search_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func_one_arch(
        valid_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.close()


[Search the 000-200-th epoch] Time Left: [00:00:00], LR=0.1
*SEARCH* [2022-09-14 12:17:43] [000-200][000/098] Time 0.64 (0.64) Data 0.33 (0.33) Base [Loss 2.352 (2.352)  Prec@1 6.25 (6.25) Prec@5 50.78 (50.78)]
*SEARCH* [2022-09-14 12:17:53] [000-200][097/098] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 1.697 (1.801)  Prec@1 35.71 (31.54) Prec@5 89.88 (84.53)]
[000-200] searching : loss=1.80, accuracy@1=31.54%, accuracy@5=84.53%, time-cost=11.2 s
[000-200] evaluate  : loss=2.09, accuracy@1=30.11%, accuracy@5=79.44%
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 000-200-th epoch : find the highest validation accuracy : 30.11%.
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 001-200-th epoch] Time Left: [01:08:23], LR=0.09999383162408304
*SEARCH* [2022-09-14 12:18:03] [001-200][000/0

*SEARCH* [2022-09-14 12:20:20] [007-200][097/098] Time 0.22 (0.14) Data 0.00 (0.00) Base [Loss 0.900 (0.952)  Prec@1 69.64 (66.00) Prec@5 94.05 (97.09)]
[007-200] searching : loss=0.95, accuracy@1=66.00%, accuracy@5=97.09%, time-cost=100.6 s
[007-200] evaluate  : loss=1.26, accuracy@1=57.24%, accuracy@5=95.80%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 007-200-th epoch : find the highest validation accuracy : 57.24%.
Find ../results/NB201/test/checkpoint/seed-16467-best.pth exist, delete is at first before saving
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 008-200-th epoch] Time Left: [01

*SEARCH* [2022-09-14 12:23:05] [015-200][097/098] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.739 (0.723)  Prec@1 74.40 (74.56) Prec@5 98.21 (98.53)]
[015-200] searching : loss=0.72, accuracy@1=74.56%, accuracy@5=98.53%, time-cost=198.9 s
[015-200] evaluate  : loss=0.95, accuracy@1=68.21%, accuracy@5=97.28%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 015-200-th epoch : find the highest validation accuracy : 68.21%.
Find ../results/NB201/test/checkpoint/seed-16467-best.pth exist, delete is at first before saving
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 016-200-th epoch] Time Left: [00

*SEARCH* [2022-09-14 12:25:50] [023-200][097/098] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 0.575 (0.602)  Prec@1 78.57 (79.05) Prec@5 99.40 (98.95)]
[023-200] searching : loss=0.60, accuracy@1=79.05%, accuracy@5=98.95%, time-cost=298.1 s
[023-200] evaluate  : loss=0.84, accuracy@1=73.08%, accuracy@5=97.68%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 023-200-th epoch : find the highest validation accuracy : 73.08%.
Find ../results/NB201/test/checkpoint/seed-16467-best.pth exist, delete is at first before saving
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 024-200-th epoch] Time Left: [00

*SEARCH* [2022-09-14 12:28:46] [032-200][097/098] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 0.689 (0.524)  Prec@1 78.57 (81.56) Prec@5 98.21 (99.21)]
[032-200] searching : loss=0.52, accuracy@1=81.56%, accuracy@5=99.21%, time-cost=406.1 s
[032-200] evaluate  : loss=1.54, accuracy@1=59.82%, accuracy@5=96.43%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 033-200-th epoch] Time Left: [00:55:10], LR=0.09343157572190958
*SEARCH* [2022-09-14 12:28:56] [033-200][000/098] Time 0.54 (0.54) Data 0.43 (0.43) Base [Loss 0.486 (0.486)  Prec@1 83.59 (83.59) Prec@5 99.22 (99.22)]
*SEARCH* [2022-09-14 12:29:08] [033-200][097/098] Time 0.14 (0.13) Data 0.00 (0.00) Base [Loss 0.603 (0.515)  Prec@1 77.98 (

...

*SEARCH* [2022-09-14 13:11:36] [158-200][000/098] Time 0.47 (0.47) Data 0.31 (0.31) Base [Loss 0.056 (0.056)  Prec@1 97.66 (97.66) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:11:51] [158-200][097/098] Time 0.13 (0.15) Data 0.00 (0.00) Base [Loss 0.104 (0.058)  Prec@1 95.24 (98.16) Prec@5 100.00 (100.00)]
[158-200] searching : loss=0.06, accuracy@1=98.16%, accuracy@5=100.00%, time-cost=1989.2 s
[158-200] evaluate  : loss=0.57, accuracy@1=86.04%, accuracy@5=99.36%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 159-200-th epoch] Time Left: [00:16:59], LR=0.01001576707564547
*SEARCH* [2022-09-14 13:12:01] [159-200][000/098] Time 0.56 (0.56) Data 0.36 (0.36) Base [Loss 0.038 (0.038)  Prec@1 9

*SEARCH* [2022-09-14 13:14:46] [167-200][000/098] Time 0.46 (0.46) Data 0.32 (0.32) Base [Loss 0.033 (0.033)  Prec@1 99.61 (99.61) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:14:56] [167-200][097/098] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.055 (0.034)  Prec@1 98.21 (99.01) Prec@5 100.00 (100.00)]
[167-200] searching : loss=0.03, accuracy@1=99.01%, accuracy@5=100.00%, time-cost=2103.0 s
[167-200] evaluate  : loss=0.55, accuracy@1=86.82%, accuracy@5=99.40%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 168-200-th epoch] Time Left: [00:09:10], LR=0.006184665997806821
*SEARCH* [2022-09-14 13:15:03] [168-200][000/098] Time 0.49 (0.49) Data 0.31 (0.31) Base [Loss 0.043 (0.043)  Prec@1 

[175-200] evaluate  : loss=0.53, accuracy@1=87.60%, accuracy@5=99.31%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 175-200-th epoch : find the highest validation accuracy : 87.60%.
Find ../results/NB201/test/checkpoint/seed-16467-best.pth exist, delete is at first before saving
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 176-200-th epoch] Time Left: [00:07:55], LR=0.0035111757055874327
*SEARCH* [2022-09-14 13:17:52] [176-200][000/098] Time 0.43 (0.43) Data 0.31 (0.31) Base [Loss 0.017 (0.017)  Prec@1 99.61 (99.61) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:18:02] [176-200][097/098] Tim

*SEARCH* [2022-09-14 13:20:43] [184-200][097/098] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 0.018 (0.015)  Prec@1 100.00 (99.69) Prec@5 100.00 (100.00)]
[184-200] searching : loss=0.02, accuracy@1=99.69%, accuracy@5=100.00%, time-cost=2314.9 s
[184-200] evaluate  : loss=0.54, accuracy@1=87.56%, accuracy@5=99.26%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 185-200-th epoch] Time Left: [00:04:35], LR=0.0013815039801161723
*SEARCH* [2022-09-14 13:20:52] [185-200][000/098] Time 0.58 (0.58) Data 0.47 (0.47) Base [Loss 0.012 (0.012)  Prec@1 100.00 (100.00) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:21:02] [185-200][097/098] Time 0.09 (0.11) Data 0.00 (0.01) Base [Loss 0.017 (0.015)  Pre

*SEARCH* [2022-09-14 13:23:21] [193-200][000/098] Time 0.48 (0.48) Data 0.32 (0.32) Base [Loss 0.010 (0.010)  Prec@1 100.00 (100.00) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:23:36] [193-200][097/098] Time 0.14 (0.16) Data 0.00 (0.00) Base [Loss 0.018 (0.013)  Prec@1 98.81 (99.79) Prec@5 100.00 (100.00)]
[193-200] searching : loss=0.01, accuracy@1=99.79%, accuracy@5=100.00%, time-cost=2420.9 s
[193-200] evaluate  : loss=0.54, accuracy@1=87.74%, accuracy@5=99.37%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 194-200-th epoch] Time Left: [00:02:12], LR=0.0002219017698460002
*SEARCH* [2022-09-14 13:23:43] [194-200][000/098] Time 0.42 (0.42) Data 0.31 (0.31) Base [Loss 0.009 (0.009)  Prec